# Motif Finding by Expectation-Maximization

EM algorithm으로 motif를 찾아본다.

생물정보학 및 실습 1, 2023년 1학기

In [ ]:
seqs = """
CGCCCCTCTCGGGGGTGTTCAG
TGTGTAAGTGCCAAGGTGCCAG
ACCGAAAGAAGTATACAGGCGT
TTCAGGTGCACGTCGGTGAACC
CTCCACGTGCAATGTTGGCCTA""".split()
seqs

In [ ]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000)

기본 파라미터 세팅

In [ ]:
W = 8 # motif width
ALPHABET = 'ACGT'
N = len(ALPHABET)
L = len(seqs[0]) # length of each sequence
pseudocount = 1

ALPHABETenum = dict(zip(ALPHABET, range(N)))

다루기 쉽게 seqs를 숫자 배열로 변환함 (A->0, C->1, G->2, T->3)

In [ ]:
X = np.array([[ALPHABETenum[c] for c in s] for s in seqs])
X

미리 전체 알파벳 빈도 계산하고 백그라운드 벡터 B 계산

In [ ]:
nc = np.array([(X == i).sum() for i in range(N)])
B = (nc + pseudocount) / (nc + pseudocount).sum()
nc, B

시작 profile을 맨 앞 부분 길이가 W인 문자열로 초기화함

In [ ]:
M = np.zeros([N, W])

for j in range(W): # for each column of motif
    for c in range(N): # for each character in the alphabet
        M[c, j] = np.mean(X[:, j] == <<1>>)

M

### E-step

Z matrix 계산

$Z_{ij}^{(t)} = \frac {\Pr(X_i | Z_{ij}=1,M^{(t)})} {\sum _{k=1} ^{L-W+1} { \Pr(X_i | Z_{ik}=1,M^{(t)})}}$

In [ ]:
Z = np.zeros([X.shape[0], L-W+1])

# 먼저 분자의 Pr(Xi...)를 계산한다.
for i in range(X.shape[0]):
    for j in range(<<2>>):
        # motif 이전 background
        pr_beforemotif = B[X[i, :j]]

        # motif 내부
        subseq = X[i, j:<<3>>]
        pr_motif = np.choose(subseq, M)

        # motif 이후 background
        pr_aftermotif = B[X[i, <<4>>:]]

        # 모두 곱해서 분모를 계산한다.
        probs = np.hstack([pr_beforemotif, pr_motif, pr_aftermotif])
        Z[i, j] = <<5>>(probs)

        # 예시 좀 보여줌
        if i < 2 and j < 5:
            print(f'z[{i},{j}] =',
                ' * '.join(format(v, 'g') for v in probs),
                '=', format(Z[i, j], 'g'))

In [ ]:
print('== 분모로 나누기 전 Z', '=' * 30)
print(Z)

# 각 sequence의 likelihood 총합을 계산해서 분모로 써서 나눈다.
Z = Z / <<6>>(axis=1)[:, np.newaxis]

print('== 분모로 나눈 후 Z', '=' * 30)
print(Z)

### M-step

`numpy`는 슬라이드 자료와 다르게 세로축을 먼저 쓰기 때문에 파라미터 순서도 k,c 대신 c,k로 바꿈.

$M^{(t+1)}(c,k) = \frac {n_{c,k} + d} {\sum _c {(n_{c,k} + d)}}$

$n_{c,k} = \sum _i \sum _{j|X_{i,j+k-1}=c} {Z_{ij}}$

In [ ]:
n = np.zeros([N, W])

# 행렬 n 먼저 계산
for c in range(N):
    for k in range(W):
        # sum 순서는 sigma i 방향을 먼저 합하는 게 간단하므로 i먼저 j나중
        # X[:, <<7>>] == c는 boolean 벡터로, X_i,j+k-1=c 조건을 담당한다.
        n[c, k] = np.sum([(Z[:, j] * (X[:, <<7>>] == c)).sum()
                          for j in range(<<8>>)])

# M 계산
M = n + pseudocount
M = M / M.sum(axis=0)
M

백그라운드 벡터 B 업데이트하기. 앞의 $n_{c,k}$와 마찬가지로 슬라이드와는 축을 반대로 써야 행렬을 표시했을 때 보기 편하다.

$B^{(t+1)}(c) = \frac {n_{c,0}+d} {\sum_c {(n_{c,0}+d)}}$

$n_{c,0}=n_c-\sum_{j=1}^W {n_{c,j}}$

In [ ]:
# nc는 시작할 때 미리 계산해 두었다.
B = nc - n.sum(<<9>>) + pseudocount
B = B / B.sum(<<10>>)
B

### 함수로 모아 두기

여러 번 돌리기 쉽게 위의 코드를 모아서 함수로 만든다.

In [ ]:
def Estep(X, B, L, W):
    Z = np.zeros([X.shape[0], <<11>>])

    for i in range(X.shape[0]):
        for j in range(L-W+1):
            pr_beforemotif = B[X[i, :j]]

            subseq = X[i, j:j+W]
            pr_motif = np.choose(subseq, M)

            pr_aftermotif = B[X[i, j+W:]]

            probs = np.hstack([pr_beforemotif, pr_motif, pr_aftermotif])
            Z[i, j] = np.prod(probs)

    return Z / Z.sum(<<12>>)[:, np.newaxis]

In [ ]:
# 서열 맨 앞부분을 모아서 만든 초기 M을 계산한다
def initialize_M(X, nc, N, W, pseudocount):
    M = np.zeros([N, W])

    for j in range(W): # for each column of motif
        for c in range(N): # for each character in the alphabet
            M[c, j] = <<13>>(X[:, j] == c)

    B = (nc + pseudocount) / (nc + pseudocount).sum()

    return M, B

In [ ]:
def Mstep(X, Z, nc, N, W, pseudocount):
    n = np.zeros([<<14>>])

    for c in range(N):
        for k in range(W):
            n[c, k] = <<15>>([(Z[:, j] * (X[:, j+k] == c)).sum()
                             for j in range(L-W+1)])

    M = n + pseudocount
    M = M / M.sum(axis=0)

    B = nc - n.sum(axis=1) + pseudocount
    B = B / B.sum(axis=0)

    return M, B

모아서 돌려보기!

In [ ]:
M, B = initialize_M(X, nc, N, W, pseudocount)
print('== (initial) M', '=' * 30)
print(M)
print('== (initial) B', '=' * 30)
print(B)

Z = Estep(X, B, L, W)
M, B = Mstep(X, Z, nc, N, W, pseudocount)
print('== (iteration 1) M', '=' * 30)
print(M)
print('== (iteration 1) B', '=' * 30)
print(B)

3번 돌려 보기

In [ ]:
M, B = initialize_M(X, nc, N, W, pseudocount)
Z = Estep(X, B, L, W)
M, B = Mstep(X, Z, nc, N, W, pseudocount)
Z = Estep(X, B, L, W)
M, B = Mstep(X, Z, nc, N, W, pseudocount)
Z = Estep(X, B, L, W)
M, B = Mstep(X, Z, nc, N, W, pseudocount)

print('== (iteration 3) M', '=' * 30)
print(M)
print('== (iteration 3) B', '=' * 30)
print(B)

delta M이 0.01% 이하가 될 때까지 E-M 반복하기

In [ ]:
deltaM_threshold = 1e-4
datalog = {'deltaM': [], 'B': [], 'Z': [], 'M': []}

M, B = initialize_M(X, nc, N, W, pseudocount)
for i in range(<<16>>): # 최대 iteration 수를 10000으로 제한한다.
    prev_M = M
    Z = Estep(X, B, L, W)
    M, B = Mstep(X, Z, nc, N, W, pseudocount)

    # M 변화를 계산하고 주요 값들을 저장한다.
    delta = (np.abs(M - <<17>>) / M).mean()
    datalog['deltaM'].append(delta)
    datalog['B'].append(B)
    datalog['Z'].append(Z)
    datalog['M'].append(M)
    if delta < ((18)):
        break

print(f'== (iteration {i+1}) M', '=' * 30)
print(M)
print(f'== (iteration {i+1}) B', '=' * 30)
print(B)

$\Delta M$ 과 B 변화를 찍어본다.

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 4))
axes[0].plot(datalog['deltaM'], marker='o', markersize=4)
axes[0].axhline(deltaM_threshold, color='r', linestyle='--')
axes[0].set_ylabel('$\Delta M$')

axes[1].plot(datalog['B'], marker='o', markersize=4, label=['A', 'C', 'G', 'T'])
axes[1].legend()
axes[1].set_ylabel('Freq. in B')
axes[1].set_xlabel('Iteration')

Z, M의 변화도 한 번 찍어본다. Z에서는 맨 앞이 선택되다가 점차 뒤로 이동해서 motif가 제대로 선택되는 것이 보인다.

In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(6, 6))

# 왼쪽에는 Z를 그림
for ax, i in zip(axes[:, 0], range(0, len(datalog['Z']), 2)):
    ax.pcolor(datalog['Z'][i], vmin=0, vmax=.5)
    plt.setp(ax.get_yticklabels() + ax.get_xticklabels(), visible=False)
    ax.set_ylabel(f'Iter. {i+1}')
    if i == 0:
        ax.set_title('Z')

# 오른쪽에 M 그림
for ax, i in zip(axes[:, 1], range(0, len(datalog['M']), 2)):
    ax.pcolor(datalog['M'][i], vmin=0.1, vmax=.4)
    plt.setp(ax.get_yticklabels() + ax.get_xticklabels(), visible=False)
    if i == 0:
        ax.set_title('M')

plt.tight_layout()

Consensus 모티프 찍어보기

In [ ]:
# M 에서 최대 확률인 염기들을 모은 consensus
consensus = ''.join(np.choose(<<19>>(axis=0), 'ACGT'))

# Z 에서 가장 확률이 높은 부분들을 고른 instances
instances = [seqs[i][j:j+W] for i, j in enumerate(<<20>>(axis=1))]

print(consensus, '\t(consensus)')
print('--' * 10)
print(*instances, sep='\n')